In [1]:
import torch
from torch import nn

In [21]:
# del new_model, resnet18, resnet50, resnet101
torch.cuda.empty_cache()

## Preprocessing

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

### Train

In [ ]:
preprocessing = transforms.Compose([
    transforms.RandomSizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize,
])

### Val

In [ ]:
preprocessing = transforms.Compose([
    transforms.Scale(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize,
])

In [ ]:
INPUT_WIDTH = 224

In [ ]:
def preprocess(np_image_bgr):
    '''
    Preprocessing for embedder network: Flips BGR to RGB, resize, convert to torch tensor, normalise with imagenet mean and variance, reshape. Note: input image yet to be loaded to GPU through tensor.cuda()

    Parameters
    ----------
    np_image_bgr : ndarray
        (H x W x C) in BGR

    Returns
    -------
    Torch Tensor

    '''
    np_image_rgb = np_image_bgr[...,::-1]
    np_image_rgb = cv2.resize(np_image_rgb, (INPUT_WIDTH, INPUT_WIDTH))
    input_image = transforms.ToTensor()(np_image_rgb)
    input_image = input_image.view(1,3,INPUT_WIDTH,INPUT_WIDTH)
    return input_image

In [4]:
from torch import nn
from pathlib import Path
import torchvision.models as models

class Resnet_Embedder(object):
    '''
    Resnet_Embedder loads a resnet pretrained on Imagenet1000, with classification layer removed, exposing the bottleneck layer, outputing a feature of size 1280. 
    '''
    def __init__(self, model_wts_path = None, depth=50):
        if model_wts_path is None:
            pretrained=True
        else:
            pretrained=False
            assert Path(model_wts_path).is_file(),'Model path does not exists!'
            
        if depth == 18:
            full_model = models.resnet18(pretrained=pretrained)
        elif depth == 50:
            full_model = models.resnet50(pretrained=pretrained)
        elif depth == 101:
            full_model = models.resnet101(pretrained=pretrained)
        else:
            assert True,'Resnet with specified depth {} does not exist!'.format(depth)

        with torch.no_grad():
            self.model = nn.Sequential(*list(full_model.children())[:-1])
            del full_model
        self.model.cuda() #loads model to gpu
        self.model.eval() #inference mode, deactivates dropout layers 
        print('Resnet{} Embedder initialised!'.format(depth))
        self.model.forward(torch.zeros(8, 3, INPUT_WIDTH, INPUT_WIDTH).cuda()) #warmup

    def predict(self, np_image_bgr):
        '''
        singly inference

        Params
        ------
        np_image_bgr : list of ndarray
            list of (H x W x C) in BGR
        
        Returns
        ------
        list of features (np.array with dim = 1280)

        '''
                img = preprocess(img)
                input_batch[k] = img
#             # Batch inference
#             # tic = time.time()
#             input_batch = input_batch.cuda()
#             # toc = time.time()
#             # print('input to cuda time:{}s'.format(toc - tic))
#             # tic = time.time()
#             output = self.model.forward(input_batch)
#             # toc = time.time()
#             # print('real inference time: {}s'.format(toc - tic))
#             all_feats.extend(output.cpu().data.numpy())
#             remainder = total_size - batch_size
#         return all_feats

In [5]:
resmodel = Resnet_Embedder(depth=50)

Resnet50 Embedder initialised!


In [ ]:
resmodel